In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
from datetime import datetime
from functools import partial

import polars as pl
import altair as alt
import pendulum


from weather.helpers.epw_read import read_epw
from weather.helpers.weather_data import PALO_ALTO_20
from weather.helpers.filter import filter_df_by_month
from weather.helpers.figures import get_todays_save_path



In [3]:
df = read_epw(PALO_ALTO_20.path)
df.head()

datetime,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Extraterrestrial Horizontal Radiation,Extraterrestrial Direct Normal Radiation,Horizontal Infrared Radiation Intensity,Global Horizontal Radiation,Direct Normal Radiation,Diffuse Horizontal Radiation,Global Horizontal Illuminance,Direct Normal Illuminance,Diffuse Horizontal Illuminance,Zenith Luminance,Wind Direction,Wind Speed,Total Sky Cover,Opaque Sky Cover (used if Horizontal IR Intensity missing),Visibility,Ceiling Height,Present Weather Observation,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
datetime[μs],f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,f64,i64,i64,i64,f64,i64,i64,f64,f64,f64
2020-01-01 00:00:00,7.2,5.6,90,0,0,298,0,0,0,0,0,0,0,320,1.2,5,5,16.1,5067,9,139,0.0,0,88,999.0,0.0,1.0
2020-01-01 01:00:00,7.0,5.5,90,0,0,297,0,0,0,0,0,0,0,320,1.0,5,5,16.1,4824,9,139,0.0,0,88,999.0,0.0,1.0
2020-01-01 02:00:00,6.8,5.4,91,0,0,296,0,0,0,0,0,0,0,160,0.8,5,5,16.1,4580,9,139,0.0,0,88,999.0,0.0,1.0
2020-01-01 03:00:00,6.6,5.3,91,0,0,295,0,0,0,0,0,0,0,160,0.6,5,5,16.1,4336,9,129,0.0,0,88,999.0,0.0,1.0
2020-01-01 04:00:00,6.4,5.2,92,0,0,294,0,0,0,0,0,0,0,160,0.4,5,5,16.1,4092,9,129,0.0,0,88,999.0,0.0,1.0


In [4]:
month_filter = partial(filter_df_by_month, df, PALO_ALTO_20)
june = month_filter(6)
july = month_filter(7)
aug = month_filter(8)

In [5]:
chart = alt.hconcat()
for df in [june, july, aug]:
    chart |= alt.Chart(df).mark_line().encode(x="datetime:T", y="Dry Bulb Temperature")

chart.resolve_scale(y="shared")

alt.HConcatChart(...)

In [15]:
def get_month(df):
    adate = pendulum.instance(df["datetime"][0])
    return adate.format("MMM")

In [24]:
pendulum.today().format("YYMMDD")

'250131'

In [27]:
chart = alt.hconcat()
for df in [june, july, aug]:
    chart |= alt.Chart(df).mark_line().encode(
        alt.X('hours(datetime):T'),
        alt.Y("Dry Bulb Temperature"),
        color='date(datetime):O').properties(title=get_month(df))
    
chart = chart.resolve_scale(y="shared")
chart.save(get_todays_save_path() / "three_month_overlay.png")
chart

alt.HConcatChart(...)

In [30]:
june["datetime"].dt.hour()

datetime
i8
0
1
2
3
4
…
20
21
22


In [34]:
time_df = pl.DataFrame
time_df.with_columns(
    am = june.filter(pl.col("datetime").dt.hour() == 6)["Dry Bulb Temperature"]
)

TypeError: DataFrame.with_columns() missing 1 required positional argument: 'self'

Dry Bulb Temperature
f64
16.2
18.2
21.2
18.2
17.1
…
17.2
18.0
17.2


In [ ]:
# now want to lay the days on top of each other.. 